## Agile Hardware Design
***
# Arbitration

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Spring21/)

## Plan for Today

* One-hot encoding
* Priority encoders
* Arbiters
* Example: crossbar

## Loading The Chisel Library Into a Notebook

In [3]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /Users/sbeamer/Spring 2021/CSE 293/lectures/08-arbit/Main.scCompiling /Users/sbeamer/Spring 2021/CSE 293/lectures/08-arbit/Main.sc #2

path: String = "/Users/sbeamer/Spring 2021/CSE 293/lectures/08-arbit/../resource/chisel_deps.sc"

In [4]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

## One-Hot Encoding

* Collection of wires where _**exactly**_ one wire is high (rest are low)

* Helpful for working with a collection of objects in which you only want one to be active/selected/enabled

* Examples
  * Setting the write enable high for the target register in register file
  * Charging the appropriate word line in a SRAM (often called a _decoder_)

* Can often avoid need to encode/decode because both producers and consumers of one-hot (OH) encoding may prefer it

<img src="images/decoder.svg" alt="xbar schematic" style="width:30%;margin-left:auto;margin-right:auto"/>

## Implementing Our Own One-Hot Encoder

In [5]:
class ConvUIntToOH(inWidth: Int) extends Module {
    val outWidth = 1 << inWidth
    val io = IO(new Bundle {
        val in  = Input(UInt(inWidth.W))
        val out = Output(UInt(outWidth.W))
    })
    require (inWidth > 0)
    def helper(index: Int): UInt = {
        if (index < outWidth) Cat(helper(index+1), io.in === index.U)
        else io.in === index.U
    }
    io.out := helper(0)
//     io.out := UIntToOH(io.in)
    printf("%d -> %b\n", io.in, io.out)
}
println(getVerilog(new ConvUIntToOH(2)))
test(new ConvUIntToOH(2)) { c =>
    for (i <- 0 until 4) {
        c.io.in.poke(i.U)
        c.io.out.expect((1 << i).U)
        c.clock.step()
    }
}

Elaborating design...
Done elaborating.
module ConvUIntToOH(
  input        clock,
  input        reset,
  input  [1:0] io_in,
  output [3:0] io_out
);
  wire [2:0] _GEN_0 = {{1'd0}, io_in}; // @[cmd4.sc 10:20]
  wire  hi = _GEN_0 == 3'h4; // @[cmd4.sc 10:20]
  wire  lo = io_in == 2'h3; // @[cmd4.sc 9:58]
  wire  lo_1 = io_in == 2'h2; // @[cmd4.sc 9:58]
  wire  lo_2 = io_in == 2'h1; // @[cmd4.sc 9:58]
  wire  lo_3 = io_in == 2'h0; // @[cmd4.sc 9:58]
  wire [4:0] _T = {hi,lo,lo_1,lo_2,lo_3}; // @[Cat.scala 30:58]
  assign io_out = _T[3:0]; // @[cmd4.sc 12:12]
  always @(posedge clock) begin
    `ifndef SYNTHESIS
    `ifdef PRINTF_COND
      if (`PRINTF_COND) begin
    `endif
        if (~reset) begin
          $fwrite(32'h80000002,"%d -> %b\n",io_in,io_out); // @[cmd4.sc 14:11]
        end
    `ifdef PRINTF_COND
      end
    `endif
    `endif // SYNTHESIS
  end
endmodule

Elaborating design...
Done elaborating.
 0 ->    1
 1 ->   10
 2 ->  100
 3 -> 1000
 0 ->    1
test ConvUIntToOH Su

defined class ConvUIntToOH

## Priority Encoder

* Given collection of wires, returns index of least significant bit that is high (1) given predefined precedence ordering (_priority_)

* Helpful for ordering logic or choosing between things

* Examples
  * In a pipelined processor to resolve RAW hazard, forward data from most recent instruction
  * In a collection components, find first free slot

* Chisel provides result as an index [`PriorityEncoder`](https://www.chisel-lang.org/api/latest/chisel3/util/PriorityEncoder\$.html),
one-hot with [`PriorityEncoderOH`](https://www.chisel-lang.org/api/latest/chisel3/util/PriorityEncoderOH\$.html), or even integrated into a Mux with [`PriorityMux`](https://www.chisel-lang.org/api/latest/chisel3/util/PriorityMux\$.html)
  * _What if input is 0?_ invalid, but returns max index or 0 (for OH)


## Example One-Hot Priority Encoders

<img src="images/priority.svg" alt="priority schematic" style="width:75%;margin-left:auto;margin-right:auto"/>

## Example One-Hot Priority Encoder Implementation

In [ ]:
class MyPriEncodeOH(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(n.W))
        val out = Output(UInt())
    })
    require (n > 0)
    def withGates(index: Int, expr: UInt): UInt = {
        if (index < (n-1)) Cat(withGates(index+1, ~io.in(index) & expr), io.in(index) & expr)
        else io.in(index) & expr
    }
    def withMuxes(index: Int): UInt = {
        if (index < n) Mux(io.in(index), (1 << index).U, withMuxes(index+1))
        else 0.U
    }
    io.out := withGates(0, 1.U)
//     io.out := withMuxes(0)
//     io.out := PriorityEncoderOH(io.in)
    printf("%b -> %b\n", io.in, io.out)
}

// println(getVerilog(new MyPriEncodeOH(2)))
test(new MyPriEncodeOH(2)) { c =>
    for (i <- 0 until 4) {
        c.io.in.poke(i.U)
        c.clock.step()
    }
}

## Arbiter

* _Arbitration_ is needed to choose between multiple components attempting to access a scarce resource

* Needs way to choose (_arbitrate_) if multiple simultaneous requests
  * If only one request, grant to lone requestor

* Different tie-breaking algorithms available e.g. fixed priority or round-robin
  * Consider needs for usage scenario

* Examples
  * Structural hazard in a processor, such as core & memory both trying to write to cache at same time
  * Output ports of a network switch (later today)

## Arbiters in Chisel

* Use `Decoupled` for both requestors and outcome
  * `valid` (from requestor) indicates if actually sending request
  * `ready` (to requestor) indicates request granted

* [`Arbiter`](https://www.chisel-lang.org/api/latest/chisel3/util/Arbiter.html) - fixed priority from least significant (e.g. port 0 wins)

* [`RRArbiter`](https://www.chisel-lang.org/api/latest/chisel3/util/RRArbiter.html) - round robin for who wins ties

* [`LockingRRArbiter`](https://www.chisel-lang.org/api/latest/chisel3/util/LockingRRArbiter.html) - round robin, but "winner" granted out for `count` cycles

<img src="images/arbiter.svg" alt="arbiter schematic" style="width:45%;margin-left:auto;margin-right:auto"/>

## Demo of Chisel util's Arbiters

In [ ]:
class UtilArbDemo(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val arb = Module(new Arbiter(UInt(n.W), numPorts))
    for (p <- 0 until numPorts) {
        arb.io.in(p) <> io.req(p) 
    }
    io.out <> arb.io.out
    printf("req: ")
    for (p <- numPorts-1 to 0 by -1) {
        printf("%b", arb.io.in(p).valid)
    }
    printf(" winner: %d (v: %b)\n", arb.io.out.bits, arb.io.out.valid)
}

// println(getVerilog(new UtilArbDemo(2,8)))
val numPorts = 4
test(new UtilArbDemo(numPorts,8)) { c =>
    c.io.out.ready.poke(true.B)
    for (cycle <- 0 until 4) {
        for (p <- 0 until numPorts) {
            c.io.req(p).bits.poke(p.U)
            c.io.req(p).valid.poke((p < 4).B)
        }
        c.clock.step()
    }
}

## Implementing Our Own Arbiter

In [ ]:
class MyArb(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val inValids = Wire(Vec(numPorts, Bool()))
    val inBits   = Wire(Vec(numPorts, UInt(n.W)))
    for (p <- 0 until numPorts) {
        io.req(p).ready := false.B
        inValids(p) := io.req(p).valid
        inBits(p) := io.req(p).bits
    }
    io.out.valid := inValids.asUInt.orR
    val chosenOH = PriorityEncoderOH(inValids)
    io.out.bits := Mux1H(chosenOH, inBits)
    val chosen = OHToUInt(chosenOH)
    when (io.out.fire) {
        io.req(chosen).ready := true.B
    }
}

## Testing Our Arbiter

In [ ]:
class ArbDemo(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val arb = Module(new MyArb(numPorts,n ))
    for (p <- 0 until numPorts) {
        arb.io.req(p) <> io.req(p) 
    }
    io.out <> arb.io.out
    printf("req: ")
    for (p <- numPorts-1 to 0 by -1) {
        printf("%b", arb.io.req(p).valid)
    }
    printf(" winner: %d (v: %b)\n", arb.io.out.bits, arb.io.out.valid)
}

// println(getVerilog(new ArbDemo(2,8)))
val numPorts = 4
test(new ArbDemo(numPorts,8)) { c =>
    c.io.out.ready.poke(true.B)
    for (cycle <- 0 until 4) {
        for (p <- 0 until numPorts) {
            c.io.req(p).bits.poke(p.U)
            c.io.req(p).valid.poke((p < 4).B)
        }
        c.clock.step()
    }
}

## Example Crossbar in Chisel

* Connects `numIns` input ports to `numOuts` output ports
  * All ports are `Decoupled`

<img src="images/xbar.svg" alt="xbar schematic" style="width:30%;margin-left:auto;margin-right:auto"/>

## Example Crossbar Implementation (1/2)

In [ ]:
class Message(numOuts: Int, length: Int) extends Bundle {
    val addr = UInt(log2Ceil(numOuts+1).W)
    val data = UInt(length.W)
    override def cloneType = (new Message(numOuts, length)).asInstanceOf[this.type]
}

class XBarIO(numIns: Int, numOuts: Int, length: Int) extends Bundle {
    val in  = Vec(numIns, Flipped(Decoupled(new Message(numOuts, length))))
    val out = Vec(numOuts, Decoupled(new Message(numOuts, length)))
    override def cloneType = (new XBarIO(numIns, numOuts, length)).asInstanceOf[this.type]
}

## Example Crossbar Implementation (2/2)

In [ ]:
class XBar(numIns: Int, numOuts: Int, length: Int) extends Module {
    val io = IO(new XBarIO(numIns, numOuts, length))
    val arbs = Seq.fill(numOuts)(Module(new RRArbiter(new Message(numOuts, length), numIns)))
    for (ip <- 0 until numIns) {
        val inReadys = Wire(Vec(numOuts, Bool()))
        for (op <- 0 until numOuts) {
            inReadys(op) := arbs(op).io.in(ip).ready
        }
        io.in(ip).ready := inReadys.asUInt.orR
    }
    for (op <- 0 until numOuts) {
        for (ip <- 0 until numIns) {
            arbs(op).io.in(ip).bits <> io.in(ip).bits
            arbs(op).io.in(ip).valid := io.in(ip).valid && (io.in(ip).bits.addr === op.U)
        }
        io.out(op) <> arbs(op).io.out
    }
    for (op <- 0 until numOuts) {
        printf(" %d -> %d (%b)", io.out(op).bits.data, op.U, io.out(op).valid)
    }
    printf("\n")
}

// println(getVerilog(new XBar(2,1,8)))

## Example Crossbar Testing

In [ ]:
val numIns = 4
val numOuts = 2
test(new XBar(numIns,numOuts,8)) { c =>
    for (ip <- 0 until numIns) {
        c.io.in(ip).valid.poke(true.B)
        c.io.in(ip).bits.data.poke(ip.U)
        c.io.in(ip).bits.addr.poke((ip % numOuts).U)
    }
    for (op <- 0 until numOuts) {
        c.io.out(op).ready.poke(true.B)
    }
    for (cycle <- 0 until 4) {
        c.clock.step()
    }
}